In [1]:
from bs4 import BeautifulSoup as BS
import urllib as url
import pandas as pd
import numpy as np

In [2]:
# Initializing Request object by passing in the earthquake's url, and assigning the reults to resulttext
request = url.request.Request("https://en.wikipedia.org/wiki/List_of_deadly_earthquakes_since_1900")
result = url.request.urlopen(request)
resulttext = result.read()

In [3]:
# Initialize a BeautifulSoup instance of the sring-ed version of the website's html
soup = BS(resulttext, 'html.parser')

In [4]:
# There are six different tables on the website, so I am targetting the table with specific classes. 
# This returns a list, so I assert that the list length is equal to one (so I know I am only working with one table),
# then assign the first index to my variable to reference.
earthquake_table_list = soup.find_all('table', class_=['sortable', 'wikitable', 'jquery-tablesorter'])
assert len(earthquake_table_list) == 1

earthquake_table = earthquake_table_list[0]

In [5]:
# Grab all of the 'th' elements, grab the text from each one, and assign the results to a list. 
# This will be used when creating the pandas df for column name assignment
table_header = earthquake_table.find_all('th')

columns = [th.text for th in table_header]

In [6]:
print(columns)

['Origin (UTC)', 'Present-day country and link to Wikipedia article', 'Lat', 'Long', 'Depth (km)', 'Magnitude', 'Secondary Effects', 'PDE Shaking Deaths', 'PDE Total Deaths', 'Utsu Total Deaths', 'EM-DAT Total Deaths', 'Other Source Deaths']


In [7]:
# Find all table rows
table_rows = earthquake_table.find_all('tr')[1:]

# Create an empty list that will ultimately hold.. more lists. The nested lists will hold each individual value per 
# column, per row
df_rows = list()
for row in table_rows:
    # Grab the text from each row, and assign it to a list. Append this list to df_rows
    row_data= [data.text for data in row.find_all('td')]
    
    df_rows.append(row_data)

In [8]:
for row in df_rows[:5]:
    print(row)

['1900-05-11 17:23', 'Japan', '38.700', '141.100', '5', '7.0 MJMA', '', '', '', '', '', '']
['1900-07-12 06:25', 'Turkey', '40.300', '43.100', '', '5.9 Muk', '', '', '', '140', '', '']
['1900-10-29 09:11', 'Venezuela', '11.000', '-66.000', '0', '7.7 Mw', '', '', '', '', '', '']
['1901-02-15 00:00', 'China', '26.000', '100.100', '0', '6.5 Ms', '', '', '', '', '', '']
['1901-03-31 07:11', 'Bulgaria', '43.400', '28.700', '', '6.4 Muk', '', '', '', '4', '', '']


In [9]:
# Create a pandas df using the columns and rows we scraped.
earthquake_df = pd.DataFrame(data=df_rows, columns=columns)

# Drop 'Other Source Deaths' column, per request from README.md
earthquake_df.drop(columns=['Other Source Deaths'], inplace=True)

earthquake_df['Magnitude'] = earthquake_df['Magnitude'].str.extract('(?P<Magnitude>\d)', expand=True)
earthquake_df['Magnitude'] = pd.to_numeric(earthquake_df['Magnitude'])

# Since the 'Origin (UTC)' column is a date string, I am transforming it to a datetime object and assigning it 
# to the index. I can imagine using datetime range queries later on, and converting these now seems appropriate
# and will be much easier later on. 
earthquake_df['Origin (UTC)'] = pd.to_datetime(earthquake_df['Origin (UTC)'])
earthquake_df.set_index('Origin (UTC)', inplace=True)

# Replace the remaining empty strings to np.nan values
# earthquake_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
earthquake_df.replace("", np.nan, inplace=True)

earthquake_df.columns

Index(['Present-day country and link to Wikipedia article', 'Lat', 'Long',
       'Depth (km)', 'Magnitude', 'Secondary Effects', 'PDE Shaking Deaths',
       'PDE Total Deaths', 'Utsu Total Deaths', 'EM-DAT Total Deaths'],
      dtype='object')

In [10]:
def nan_to_int(df, columns, value=-1, inplace=True):
    """Transforms the column in the dataframe to a float32 column, rather than the object type. 
    Also fills all NaN values to -1.
    """
    for col in columns:        
        df[col].fillna(value=value, inplace=inplace)
        df[col] = pd.to_numeric(df[col], downcast='float')

In [11]:
"""There are some bad characters in this column, so this regex just grabs the int"""
earthquake_df['EM-DAT Total Deaths'] = earthquake_df['EM-DAT Total Deaths'].str.extract('(\d+)', expand=True)

In [12]:
columns_to_convert = ['PDE Total Deaths', 'PDE Shaking Deaths', 'Utsu Total Deaths', 'EM-DAT Total Deaths', 'Magnitude']
nan_to_int(df=earthquake_df, columns=columns_to_convert)

In [13]:
earthquake_df.head()

,Present-day country and link to Wikipedia article,Lat,Long,Depth (km),Magnitude,Secondary Effects,PDE Shaking Deaths,PDE Total Deaths,Utsu Total Deaths,EM-DAT Total Deaths
Origin (UTC),,,,,,,,,,
1900-05-11 17:23:00,Japan,38.700,141.100,5,7.0,NaN,-1.0,-1.0,-1.0,-1.0
1900-07-12 06:25:00,Turkey,40.300,43.100,NaN,5.0,NaN,-1.0,-1.0,140.0,-1.0
1900-10-29 09:11:00,Venezuela,11.000,-66.000,0,7.0,NaN,-1.0,-1.0,-1.0,-1.0
1901-02-15 00:00:00,China,26.000,100.100,0,6.0,NaN,-1.0,-1.0,-1.0,-1.0
1901-03-31 07:11:00,Bulgaria,43.400,28.700,NaN,6.0,NaN,-1.0,-1.0,4.0,-1.0


In [14]:
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1340 entries, 1900-05-11 17:23:00 to 2011-09-23 10:41:00
Data columns (total 10 columns):
Present-day country and link to Wikipedia article    1340 non-null object
Lat                                                  1326 non-null object
Long                                                 1326 non-null object
Depth (km)                                           1250 non-null object
Magnitude                                            1340 non-null float32
Secondary Effects                                    373 non-null object
PDE Shaking Deaths                                   1340 non-null float32
PDE Total Deaths                                     1340 non-null float32
Utsu Total Deaths                                    1340 non-null float32
EM-DAT Total Deaths                                  1340 non-null float32
dtypes: float32(5), object(5)
memory usage: 89.0+ KB
